In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r'DOH COVID Data Drop_ 20201023 - 04 Case Information.csv')
data

In [ ]:
data.info()

In [ ]:
df = data.drop('CaseCode', axis=1)
df = df.drop(df.loc[:,'AgeGroup':'Sex'].columns, axis=1)
df = df.drop(df.loc[:, 'DateResultRelease':'Admitted'].columns, axis=1)
df = df.drop('ProvRes', axis=1)
df = df.drop(df.loc[:, 'CityMuniPSGC':'BarangayPSGC'].columns, axis=1)
df = df.drop(df.loc[:, 'DateOnset':'ValidationStatus'].columns, axis=1)
df

In [ ]:
titles = list(df.columns)
titles

In [ ]:
titles[4], titles[5] = titles[5], titles[4]
titles[2], titles[3] = titles[3], titles[2]
titles

In [ ]:
df = df[titles]
df

In [ ]:
start_date = '2020-09-10'
end_date = '2020-10-10'

df = df.loc[df['RegionRes'] == 'NCR']
df = df.loc[(df['DateSpecimen'] >= start_date) & ((df['DateSpecimen'] <= end_date))]
df

In [ ]:
#Cities in NCR
cities = df['CityMunRes'].unique().tolist()

cities

In [ ]:
#df = df.loc[(df['CityMunRes'] == 'QUEZON CITY') | (df['CityMunRes'] == 'CITY OF MAKATI')]
df = df.reset_index(drop=True)
df

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)
df

In [ ]:
df['DateSpecimen'] = pd.to_datetime(df['DateSpecimen'], format='%Y-%m-%d')
df

In [ ]:
df['DateSpecimen'] = df['DateSpecimen'].dt.strftime('%m/%d/%Y')
df

In [ ]:
qc = df.loc[df['CityMunRes'] == 'QUEZON CITY']
mkt = df.loc[df['CityMunRes'] == 'CITY OF MAKATI']

qc = qc.reset_index(drop=True)
mkt = mkt.reset_index(drop=True)

In [ ]:
qc

In [ ]:
mkt

In [ ]:
#1st City = QC
qcHealth = qc['HealthStatus'].unique().tolist()
qcDate = qc['DateSpecimen'].unique().tolist()
qcQuar = qc['Quarantined'].unique().tolist()

#2nd City = MKT
mktHealth = mkt['HealthStatus'].unique().tolist()
mktDate = mkt['DateSpecimen'].unique().tolist()
mktQuar = mkt['Quarantined'].unique().tolist()
mktCity = mkt['CityMunRes'].unique().tolist()
mktReg = mkt['RegionRes'].unique().tolist()

#Overall
uniHealth = df['HealthStatus'].unique().tolist()
uniDate = df['DateSpecimen'].unique().tolist()
uniQuar = df['Quarantined'].unique().tolist()
uniCity = df['CityMunRes'].unique().tolist()

In [ ]:
#1st City = QC
for i in range(len(qc)):
    for l in qcHealth:
        if qc['HealthStatus'][i] == l:
            qc['HealthStatus'][i] = qcHealth.index(l)
            
    for m in qcDate:
        if qc['DateSpecimen'][i] == m:
            qc['DateSpecimen'][i] = qcDate.index(m)
    
    for n in qcQuar:
        if qc['Quarantined'][i] == n:
            qc['Quarantined'][i] = qcQuar.index(n)
   
#2nd City = Mkt
for i in range(len(mkt)):
    for l in mktHealth:
        if mkt['HealthStatus'][i] == l:
            mkt['HealthStatus'][i] = mktHealth.index(l)
            
    for m in uniDate:
        if mkt['DateSpecimen'][i] == m:
            mkt['DateSpecimen'][i] = mktDate.index(m)
    
    for n in mktQuar:
        if mkt['Quarantined'][i] == n:
            mkt['Quarantined'][i] = mktQuar.index(n)

    for o in mktCity:
        if mkt['CityMunRes'][i] == o:
            mkt['CityMunRes'][i] = mktCity.index(o)
    
    for p in mktReg:
        if mkt['RegionRes'][i] == p:
            mkt['RegionRes'][i] = mktReg.index(p)

In [ ]:
#Overall
for i in range(len(df)):
    for l in uniHealth:
        if df['HealthStatus'][i] == l:
            df['HealthStatus'][i] = uniHealth.index(l)
            
    for m in uniDate:
        if df['DateSpecimen'][i] == m:
            df['DateSpecimen'][i] = uniDate.index(m)

    for n in uniQuar:
        if df['Quarantined'][i] == n:
            df['Quarantined'][i] = uniQuar.index(n)
    
    for o in uniCity:
        if df['CityMunRes'][i] == o:
            df['CityMunRes'][i] = uniCity.index(o)

In [ ]:
mkt

In [ ]:
qc[['HealthStatus', 'DateSpecimen', 'Quarantined']] = qc[['HealthStatus', 'DateSpecimen', 'Quarantined']].apply(pd.to_numeric)
mkt[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']] = mkt[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']].apply(pd.to_numeric)

#combined
df[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes']] = df[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes']].apply(pd.to_numeric)

In [ ]:
mkt.corr(method='pearson')

In [ ]:
cols = mkt.loc[:, 'Age':'Quarantined'].columns.to_list()
x = mkt[cols]
y = mkt['HealthStatus']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
#logreg = LogisticRegression()
#logreg2 = LogisticRegression()
clf = DecisionTreeClassifier()

In [ ]:
#logreg.fit(x_train, y_train)
#logreg2.fit(x_train2, y_train2)
clf = clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
matrix = metrics.confusion_matrix(y_test, y_pred)
matrix

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.heatmap(pd.DataFrame(matrix), annot=True, cmap="coolwarm",fmt='g')
ax.xaxis.set_label_position("top")
ax.set_xticklabels([mktHealth[0],mktHealth[1],mktHealth[2],mktHealth[3],mktHealth[4],mktHealth[5]])
ax.set_yticklabels([mktHealth[0],mktHealth[1],mktHealth[2],mktHealth[3],mktHealth[4],mktHealth[5]])
plt.title('Matrix')
plt.show()